In [1]:
#Necessary packages
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from math import sqrt
import datetime 
import calendar
import sys
import matplotlib.pyplot as plt

In [ ]:
#Load Data
# Maize silage price data
file_path = "C:/Users/PHIRI003/OneDrive - Wageningen University & Research/Documents/WEcR Internship/Work/Dairy/MaizeSilage.xlsx"

silage_df= pd.read_excel(file_path, header=0)

#Mapping month to numeric
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

silage_df['Month'] = silage_df['Month'].map(month_mapping)

#Create date column
silage_df['Date'] = pd.to_datetime(silage_df[['Month', 'Year']].assign(DAY=1))

#Order of columns
cols = ['Date'] + [col for col in silage_df if col != 'Date']
silage_df = silage_df[cols]

#Drop unnecessary columns
#col_drop = ['Year', 'Month']
#cao_df = cao_df.drop(columns=col_drop)

silage_df.head()

In [34]:
#LOading other datasets
# Dairy products data (farm (API), processing (PPI) and retail(CPI))
file_path2 = "C:/Users/PHIRI003/OneDrive - Wageningen University & Research/Documents/WEcR Internship/Work/Dairy/Dairy_Data.xlsx"
Dairy_df= pd.read_excel(file_path2, header=0, sheet_name = "Dairy")

nT = len(Dairy_df)  #date range lenth

# Generate the date range starting from '2005-01-01'
date_range = pd.date_range(start='2005-01-01', periods=nT, freq='M')

# Assign the date range to a new column in the DataFrame
Dairy_df['Date'] = date_range
Dairy_df['Date'] = pd.to_datetime(Dairy_df['Date']) # to datetime
Dairy_df['Date'] = Dairy_df['Date'].dt.to_period('M').dt.to_timestamp() #First day of the month

# Cost of labour data, average wages
file_path3 = "C:/Users/PHIRI003/OneDrive - Wageningen University & Research/Documents/WEcR Internship/Work/Dairy/cao_data_2020.xlsx"
cao_df= pd.read_excel(file_path3, header=0)
cao_df['Date'] = pd.to_datetime(cao_df['Date'])


In [ ]:
print(Dairy_df.head())
print(silage_df.head())
print(cao_df.head())

In [36]:
df_merged = pd.merge(silage_df, cao_df, on='Date', suffixes=('_si', '_ca'))

print(df_merged.head())

        Date  Year_si  Month_si  Maize  Year_ca  Month_ca  AWage  CWage  \
0 2023-10-01     2023        10   91.5     2023        10  133.6  137.7   
1 2023-09-01     2023         9   94.5     2023         9  133.2  137.3   
2 2023-08-01     2023         8  106.5     2023         8  132.8  136.9   
3 2023-07-01     2023         7  108.0     2023         7  132.6  136.6   
4 2023-06-01     2023         6  107.0     2023         6  130.8  134.8   

     AWage_20    CWage_20  
0  113.380481  113.535798  
1  113.041018  113.205991  
2  112.701556  112.876185  
3  112.531825  112.628831  
4  111.004243  111.144702  


In [37]:
# Adding the cao_df 
df_merged = pd.merge(df_merged, Dairy_df, on='Date')

In [38]:
print(df_merged.head())

        Date  Year_si  Month_si  Maize  Year_ca  Month_ca  AWage  CWage  \
0 2023-10-01     2023        10   91.5     2023        10  133.6  137.7   
1 2023-09-01     2023         9   94.5     2023         9  133.2  137.3   
2 2023-08-01     2023         8  106.5     2023         8  132.8  136.9   
3 2023-07-01     2023         7  108.0     2023         7  132.6  136.6   
4 2023-06-01     2023         6  107.0     2023         6  130.8  134.8   

     AWage_20    CWage_20  Unnamed: 0         API         PPI         CPI  \
0  113.380481  113.535798      202310  118.159359  158.575727  127.639507   
1  113.041018  113.205991      202309  118.876069  157.873621  129.211423   
2  112.701556  112.876185      202308  124.674902  157.472417  131.525631   
3  112.531825  112.628831      202307  124.674902  158.676028  132.756964   
4  111.004243  111.144702      202306  124.674902  155.867603  135.175967   

        Wheat         Oil  
0  116.683861  216.958352  
1  123.651976  224.429766  
2 

In [40]:
col_drop = ['Year_si', 'Month_si', 'Year_ca',  'Month_ca',  'AWage', 'CWage', 'Unnamed: 0',  'Wheat']
df_merged = df_merged.drop(columns=col_drop)
print(df_merged.head())

        Date  Maize    AWage_20    CWage_20         API         PPI  \
0 2023-10-01   91.5  113.380481  113.535798  118.159359  158.575727   
1 2023-09-01   94.5  113.041018  113.205991  118.876069  157.873621   
2 2023-08-01  106.5  112.701556  112.876185  124.674902  157.472417   
3 2023-07-01  108.0  112.531825  112.628831  124.674902  158.676028   
4 2023-06-01  107.0  111.004243  111.144702  124.674902  155.867603   

          CPI         Oil  
0  127.639507  216.958352  
1  129.211423  224.429766  
2  131.525631  206.302010  
3  132.756964  191.838119  
4  135.175967  179.218136  


In [47]:
file_path4 = "C:/Users/PHIRI003/OneDrive - Wageningen University & Research/Documents/WEcR Internship/Work/Dairy/Corn_Prices.xlsx"
corn_df= pd.read_excel(file_path4, header=0, sheet_name = 'Data')

print(corn_df.head())

        Date   CornPrice
0 2005-01-01   96.038759
1 2005-02-01   94.359394
2 2005-03-01  100.014011
3 2005-04-01   96.470872
4 2005-05-01   95.364812


In [48]:
corn_df['Date'] = pd.to_datetime(corn_df['Date'])

Dairy_Merged  = pd.merge(df_merged, corn_df, on='Date')
print(Dairy_Merged.head())

        Date  Maize    AWage_20    CWage_20         API         PPI  \
0 2023-10-01   91.5  113.380481  113.535798  118.159359  158.575727   
1 2023-09-01   94.5  113.041018  113.205991  118.876069  157.873621   
2 2023-08-01  106.5  112.701556  112.876185  124.674902  157.472417   
3 2023-07-01  108.0  112.531825  112.628831  124.674902  158.676028   
4 2023-06-01  107.0  111.004243  111.144702  124.674902  155.867603   

          CPI         Oil   CornPrice  
0  127.639507  216.958352  221.902519  
1  129.211423  224.429766  223.847666  
2  131.525631  206.302010  207.675889  
3  132.756964  191.838119  235.271947  
4  135.175967  179.218136  266.935240  


In [51]:
# Filter the DataFrame to include only rows from the year 2020
df_2020 = Dairy_Merged[Dairy_Merged['Date'].dt.year == 2020]

# Calculate the average for a specific variable, e.g., 'Maize'
average_maize_2020 = df_2020['Maize'].mean()
average_corn_2020 = df_2020['CornPrice'].mean()

print(average_maize_2020)
print(average_corn_2020)

72.25
165.578388484418


In [53]:
#Calculating price index for corn and silage prices
Dairy_Merged['Silage_PI'] = (Dairy_Merged['Maize']/average_maize_2020)*100
Dairy_Merged['Corn_PI'] = (Dairy_Merged['CornPrice']/average_corn_2020)*100
print(Dairy_Merged.head())

        Date  Maize    AWage_20    CWage_20         API         PPI  \
0 2023-10-01   91.5  113.380481  113.535798  118.159359  158.575727   
1 2023-09-01   94.5  113.041018  113.205991  118.876069  157.873621   
2 2023-08-01  106.5  112.701556  112.876185  124.674902  157.472417   
3 2023-07-01  108.0  112.531825  112.628831  124.674902  158.676028   
4 2023-06-01  107.0  111.004243  111.144702  124.674902  155.867603   

          CPI         Oil   CornPrice   Silage_PI     Corn_PI  
0  127.639507  216.958352  221.902519  126.643599  134.016595  
1  129.211423  224.429766  223.847666  130.795848  135.191355  
2  131.525631  206.302010  207.675889  147.404844  125.424514  
3  132.756964  191.838119  235.271947  149.480969  142.090975  
4  135.175967  179.218136  266.935240  148.096886  161.213817  


In [54]:
Dairy_Merged.to_excel('C:/Users/PHIRI003/OneDrive - Wageningen University & Research/Documents/WEcR Internship/Work/Dairy/Merged_data.xlsx', index=False)